In [9]:
import bw2data as bd
import bw2calc as bc

In [10]:
bd.projects.set_current("bw25_plca_grid_expansion")

## Components to materials


In [11]:
bd.databases

Databases dictionary with 17 object(s):
	ecoinvent-3.10-biosphere
	ecoinvent-3.10-cutoff
	ei310_SSP2_RCP26_2023
	ei310_SSP2_RCP26_2025
	ei310_SSP2_RCP26_2030
	ei310_SSP2_RCP26_2035
	ei310_SSP2_RCP26_2037
	ei310_SSP2_RCP26_2040
	grid_components_2023
	grid_components_2025
	grid_components_2030
	grid_components_2035
	grid_components_2037
	grid_components_2040
	grid_expansion_prospective
	grid_expansion_static
	grid_status_quo

In [14]:
status_quo_node = bd.get_node(database="grid_status_quo", name="grid_status_quo")
fu = {status_quo_node: 1}

In [15]:
method = (
    "EF v3.1 no LT",
    "climate change no LT",
    "global warming potential (GWP100) no LT",
)

In [16]:
component_scores = {}
lca = bc.LCA(fu, method=method)
lca.lci(factorize=True)
for component in status_quo_node.technosphere():
    material_scores = {}
    for material in component.input.technosphere():
        lca.lcia(demand={material.input.id: material.amount * component.amount})
        material_scores[material.input["name"]] = lca.score
    component_scores[component.input["name"]] = material_scores

/Users/timodiepers/anaconda3/envs/timex/lib/python3.11/site-packages/bw2calc/lca_base.py:127: SparseEfficiencyWarning: splu converted its input to CSC format
  self.solver = factorized(self.technosphere_matrix)
/Users/timodiepers/anaconda3/envs/timex/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 1.51e+12)
  warnings.warn(msg, UmfpackWarning)


In [17]:
import pandas as pd

df = pd.DataFrame(component_scores)

In [21]:
def map_labels(labels, groups, other_label="other"):
    new_labels = {}
    for label in labels:
        for key, group in groups.items():
            if key in label:
                new_labels[label] = group
                break
        else:
            new_labels[label] = other_label
    return new_labels

In [22]:
component_groups = {
    "Overhead line": "overhead lines",
    "cable": "cables",
    "Transformer": "transformers",
    "switchgear": "switchgears",
    "Substation": "substations",
}

material_groups = {
    "aluminium": "aluminium",
    "copper": "copper",
    "iron": "iron & steel",
    "steel": "iron & steel",
    "concrete": "concrete",
    "cement": "concrete",
    "sulfur hexafluoride": "SF6",
    "polyethylene": "plastics",
    "polypropylene": "plastics",
    "plastic": "plastics",
}

df.index = df.index.map(
    map_labels(df.index, material_groups, other_label="other materials")
)
df.columns = df.columns.map(map_labels(df.columns, component_groups))

df_components_to_materials = df.groupby(level=0).sum().T.groupby(level=0).sum().T
df_components_to_materials

,cables,overhead lines,substations,switchgears,transformers
SF6,0.000000e+00,0.000000e+00,0.000000e+00,5.091213e+08,0.000000e+00
aluminium,1.521202e+10,1.711174e+10,0.000000e+00,3.240757e+08,8.015168e+08
concrete,0.000000e+00,2.830048e+09,2.124849e+08,0.000000e+00,0.000000e+00
copper,2.908041e+09,1.473018e+09,0.000000e+00,4.740770e+07,5.231059e+08
iron & steel,4.013115e+08,9.674725e+09,7.983608e+08,3.937028e+07,1.494676e+09
other materials,4.074928e+08,6.109410e+08,0.000000e+00,5.853176e+07,6.963887e+08
plastics,5.495226e+09,8.901742e+06,0.000000e+00,0.000000e+00,0.000000e+00


## Materials to processes


In [23]:
aggregated_material_names = [
    "aggregated material: aluminium",
    "aggregated material: steel",
    "aggregated material: concrete",
    "aggregated material: copper",
    "aggregated material: plastics",
    "aggregated material: other",
]

aggregated_material_nodes = [
    bd.get_node(name=name) for name in aggregated_material_names
]

In [24]:
import bw2analyzer as ba

In [25]:
lca = bc.LCA(
    {aggregated_material_nodes[0]: 1}, method=method
)  # just to build the matrices
lca.lci(factorize=True)
material_top_processes = {}
for mat in aggregated_material_nodes:
    lca.lcia(demand={mat.id: 1})
    top_processes = ba.ContributionAnalysis().annotated_top_processes(lca, limit=10000)
    score_and_product = {}
    for process in top_processes:
        if process[2]["reference product"] in score_and_product:
            score_and_product[process[2]["reference product"]] += process[0]
        else:
            score_and_product[process[2]["reference product"]] = process[0]
    material_top_processes[mat["name"]] = score_and_product

/Users/timodiepers/anaconda3/envs/timex/lib/python3.11/site-packages/bw2calc/lca_base.py:127: SparseEfficiencyWarning: splu converted its input to CSC format
  self.solver = factorized(self.technosphere_matrix)
/Users/timodiepers/anaconda3/envs/timex/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 1.51e+12)
  warnings.warn(msg, UmfpackWarning)


KeyError: 

In [12]:
df_aggregated_materials = pd.DataFrame(material_top_processes)

In [14]:
product_groups = {
    "electricity": "electricity",
    "heat": "heat",
    "transport": "transport",
    "aluminium": "aluminium (process emissions)",
    "iron": "iron & steel (process emissions)",
    "steel": "iron & steel (process emissions)",
    "coal": "coal",
    "coke": "coal",
    "clinker": "clinker",
    "diesel": "transport",
    "Gas insulated switchgear": "SF6",
    "sulfur hexafluoride": "SF6",
}

df_aggregated_materials.index = df_aggregated_materials.index.map(
    map_labels(
        df_aggregated_materials.index, product_groups, other_label="other processes"
    )
)
df_aggregated_materials.columns = df_aggregated_materials.columns.map(
    map_labels(
        df_aggregated_materials.columns, material_groups, other_label="other materials"
    )
)

df_materials_to_processes = (
    df_aggregated_materials.groupby(level=0).sum().T.groupby(level=0).sum().T
)
df_materials_to_processes

,aluminium,concrete,copper,iron & steel,other materials,plastics
SF6,1.328250e+06,1.767260e+04,1.056766e+05,7.774081e+04,4.897267e+08,8.406842e+04
aluminium (process emissions),4.295369e+09,1.664928e+06,2.072173e+07,4.001359e+06,2.298309e+06,4.520097e+06
clinker,8.431941e+07,1.868708e+09,5.153069e+07,2.261534e+08,2.152376e+07,2.789748e+07
coal,4.000116e+09,9.731506e+07,2.852101e+08,2.390882e+09,6.267419e+07,2.953253e+08
electricity,2.307814e+10,3.673784e+08,2.515696e+09,1.863488e+09,5.490133e+08,1.589289e+09
heat,5.711216e+09,6.267827e+07,5.193807e+08,4.805306e+08,2.717894e+08,2.961497e+08
iron & steel (process emissions),1.497928e+08,2.516212e+07,6.049658e+07,6.719249e+09,2.550158e+07,6.064440e+07
other processes,1.644958e+09,1.900728e+08,6.804347e+08,9.294676e+08,6.605606e+08,2.913504e+09
transport,1.272192e+09,4.862868e+08,4.454903e+08,1.067972e+09,2.002017e+08,3.149924e+08


In [15]:
df_materials_to_processes.aluminium.sum()

40237427842.16292

## Prepare sankey diagram


In [16]:
sankey_data = []  # will be filled with tuples of (source, target, value)

Total impact:


In [17]:
df_components_to_materials.sum().sum()

69383294328.312

Grid to components:


In [18]:
for component, score in df_components_to_materials.sum().items():
    sankey_data.append((component, "grid status quo", score))

Components to materials:


In [19]:
for target, values in df_components_to_materials.items():
    for source, value in values.items():
        if value != 0:
            sankey_data.append((source, target, value))

Materials to processes:


In [20]:
for target, values in df_materials_to_processes.items():
    for source, value in values.items():
        if value != 0:
            sankey_data.append((source, target, value))

Create a df and add direct emissions


In [28]:
df_sankey = pd.DataFrame(sankey_data, columns=["source", "target", "value"])

In [29]:
df_sankey.value = df_sankey.value / 1e9

In [30]:
df_sankey

,source,target,value
0,cables,grid status quo,26.962429
1,overhead lines,grid status quo,36.428166
2,substations,grid status quo,1.108168
3,switchgears,grid status quo,1.046567
4,transformers,grid status quo,3.837964
...,...,...,...
76,electricity,plastics,1.589289
77,heat,plastics,0.296150
78,iron & steel (process emissions),plastics,0.060644
79,other processes,plastics,2.913504


In [31]:
df_sankey.to_csv("sankey_data.csv", index=False)